# Day 11: モルフォロジー処理
## Learning Objectives
- モルフォロジー演算を理解する
- 2値画像処理を習得する
- オープニング・クロージングを理解する

## モルフォロジー処理の基礎
**モルフォロジーとは**:
- 2値画像（白黒）に対する形状変形
- 構造要素（カーネル）を使用
- ノイズ除去、形状解析に使用

In [ ]:
# 構造要素の定義
def create_square_kernel(size=3):
    """正方形構造要素を作成"""
    return [[1 for _ in range(size)] for _ in range(size)]

def create_cross_kernel(size=3):
    """十字型構造要素を作成"""
    kernel = [[0 for _ in range(size)] for _ in range(size)]
    center = size // 2
    for i in range(size):
        kernel[i][center] = 1
        kernel[center][i] = 1
    return kernel

def create_disk_kernel(radius=1):
    """円形構造要素を作成"""
    size = 2 * radius + 1
    kernel = [[0 for _ in range(size)] for _ in range(size)]
    center = radius
    for i in range(size):
        for j in range(size):
            if (i - center)**2 + (j - center)**2 <= radius**2:
                kernel[i][j] = 1
    return kernel

# 構造要素を表示
print("3x3正方形構造要素:")
for row in create_square_kernel(3):
    print(row)

print("\n十字型構造要素:")
for row in create_cross_kernel(3):
    print(row)

print("\n円形構造要素 (半径=1):")
for row in create_disk_kernel(1):
    print(row)

## 基本演算

In [ ]:
def erode(image, struct_element):
    """収縮演算

    白領域を1ピクセル分縮小
    """
    rows = len(image)
    cols = len(image[0])
    result = [[0 for _ in range(cols)] for _ in range(rows)]

    # 構造要素のサイズ
    k_size = len(struct_element)
    padding = k_size // 2

    for i in range(padding, rows - padding):
        for j in range(padding, cols - padding):
            # 構造要素との一致をチェック
            match = True
            for ki in range(-padding, padding + 1):
                for kj in range(-padding, padding + 1):
                    if struct_element[ki + padding][kj + padding] == 1:
                        # 構造要素が1の位置で、画像が0なら不一致
                        if image[i + ki][j + kj] == 0:
                            match = False
                            break
                if not match:
                    break
            
            # 一致したら白（1）、不一致なら黒（0）
            result[i][j] = 1 if match else 0

    return result

def dilate(image, struct_element):
    """膨張演算

    白領域を1ピクセル分拡大
    """
    rows = len(image)
    cols = len(image[0])
    result = [[0 for _ in range(cols)] for _ in range(rows)]

    # 構造要素のサイズ
    k_size = len(struct_element)
    padding = k_size // 2

    for i in range(padding, rows - padding):
        for j in range(padding, cols - padding):
            # 構造要素との一致をチェック
            match = False
            for ki in range(-padding, padding + 1):
                for kj in range(-padding, padding + 1):
                    if struct_element[ki + padding][kj + padding] == 1:
                        # 構造要素が1の位置で、画像が1なら一致
                        if image[i + ki][j + kj] == 1:
                            match = True
                            break
                if match:
                    break
            
            # 一致したら白（1）、不一致なら黒（0）
            result[i][j] = 1 if match else 0

    return result

# テスト画像（白領域が十字型）
test_image = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

def print_binary_image(image, title):
    print(f"\n{title}:")
    for row in image:
        print(' '.join(['1' if x == 1 else '0' for x in row]))

print_binary_image(test_image, "元の画像")

# 収縮と膨張を適用
kernel = create_square_kernel(3)
eroded = erode(test_image, kernel)
dilated = dilate(test_image, kernel)

print_binary_image(eroded, "収縮後")
print_binary_image(dilated, "膨張後")

## 複合演算（オープニングとクロージング）

In [ ]:
def opening(image, struct_element):
    """オープニング演算

    収縮 → 膨張
    外部ノイズ除去に効果的
    """
    return dilate(erode(image, struct_element), struct_element)

def closing(image, struct_element):
    """クロージング演算

    膨張 → 収縮
    内部ノイズ除去に効果的
    """
    return erode(dilate(image, struct_element), struct_element)

def hit_or_miss(image, struct_element):
    """ヒット・オア・ミス変換

    特定の形状を検出
    """
    rows = len(image)
    cols = len(image[0])
    result = [[0 for _ in range(cols)] for _ in range(rows)]

    # 構造要素のサイズ
    k_size = len(struct_element)
    padding = k_size // 2

    for i in range(padding, rows - padding):
        for j in range(padding, cols - padding):
            # 構造要素の前景と背景をチェック
            match = True
            for ki in range(-padding, padding + 1):
                for kj in range(-padding, padding + 1):
                    if struct_element[ki + padding][kj + padding] == 1:
                        if image[i + ki][j + kj] != 1:
                            match = False
                            break
                    elif struct_element[ki + padding][kj + padding] == 0:
                        if image[i + ki][j + kj] != 0:
                            match = False
                            break
                if not match:
                    break
            
            result[i][j] = 1 if match else 0

    return result

# オープニングとクロージングを適用
opened = opening(test_image, kernel)
closed = closing(test_image, kernel)

print_binary_image(opened, "オープニング後")
print_binary_image(closed, "クロージング後")

print("特徴:")
print("- オープニング: 外部ノイズ除去、物体間の分離")
print("- クロージング: 内部ノイズ除去、物体間の結合")

## Exercise: モルフォロジー応用

In [ ]:
def remove_noise(image, struct_element, iterations=1):
    """ノイズ除去用モルフォロジー処理

    オープニングとクロージングを交互に適用
    """
    result = image
    for _ in range(iterations):
        result = opening(closing(result, struct_element), struct_element)
    return result

# ノイズ付き画像を作成
noisy_image = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0]
]

# 外部ノイズ（孤立ピクセル）を追加
noisy_image[0][0] = 1
noisy_image[0][6] = 1
noisy_image[6][0] = 1
noisy_image[6][6] = 1
noisy_image[3][3] = 0  # 内部ノイズ（穴）を作成

print("ノイズ付き画像:")
print_binary_image(noisy_image, "ノイズ付き")

# ノイズ除去
denoised = remove_noise(noisy_image, kernel)
print("\nノイズ除去後:")
print_binary_image(denoised, "除去後")

print("\n応用例:")
print("- 文字認識におけるノイズ除去")
print("- 医療画像の前処理")
print("- 工業製品の欠陥検出")

## Exercise: 骨格化処理

In [ ]:
def skeletonize(image, iterations=10):
    """極端な収縮による骨格化

    物体の形状を骨格として抽出
    """
    result = image
    for _ in range(iterations):
        # 反復収縮
        result = erode(result, kernel)
        # 収縮できなくなったら終了
        if sum(sum(row) for row in result) == 0:
            break
    return result

# 骨格化
skeleton = skeletonize(test_image)
print("\n骨格化:")
print_binary_image(skeleton, "骨格化後")

print("\n骨格化の用途:")
print("- 手書き文字の細線化")
print("- 血管の抽出")
print("- 網羅構造の分析")

**お疲れ様でした！** Day 11完了！

## Self-Check
- [ ] 収縮と膨胀の原理を理解した
- [ ] オープニングとクロージングを実装した
- [ ] ノイズ除去にモルフォロジーを適用した
- [ ] 骨格化の概念を理解した

**次回（Day 12）からは特徴抽出と領域分割を学びます。**